### First we read the data and clean it

In [2]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pyarabic.araby import strip_tashkeel


Setup

In [13]:

file_path = './Dataset/train.txt'

# Read Arabic text from the file
with open(file_path, 'r', encoding='utf-8') as file:
    arabic_text_from_file = file.read()

#print(arabic_text_from_file)

#Split the text into words
arabic_words = arabic_text_from_file.split()

#remove brackets, commas, dots, numbers using regex
arabic_words = [re.sub(r'[^\u0600-\u0660\.]+', '', word) for word in arabic_words]
stop_symbols = ['.', '،', '؟', '؛', '!', ':','...', '?.']
arabic_words = [word for word in arabic_words if word and len(word) > 1 and word not in stop_symbols]

In [8]:
with open('./Dataset/words.txt', 'w', encoding='utf-8') as file:
    for word in arabic_words:
        file.write(word + '\n')

#join the words into a string and split them according to sentences that end with a dot, question mark, or exclamation mark
arabic_sentences = ' '.join(arabic_words)
arabic_sentences = re.split(r'[\.\u061A-\u061F]', arabic_sentences) 
arabic_sentences = [sentence for sentence in arabic_sentences if sentence]

with open('./Dataset/sentences.txt', 'w', encoding='utf-8') as file:
    for sentence in arabic_sentences:
        file.write(sentence + '\n')

# with open('./Dataset/lolTest.txt', 'w', encoding='utf-8') as file:
#     for words in arabic_words:
#         if words in stop_symbols:
#             file.write(words + '\n')  

Words without tashkeel

In [17]:
arabic_words_without_punc = [re.sub(r'([^\u0600-\u0660\.]+)|[،\.\u061A-\u061F]+', '', word) for word in arabic_words]
arabic_words_without_punc = [word for word in arabic_words_without_punc if word and len(word) > 1 and word not in stop_symbols]

with open('./Dataset/WordsWithoutTashkeel.txt', 'w', encoding='utf-8') as output_file:
    for word in arabic_words_without_punc:
        output_file.write(strip_tashkeel(word) + '\n')
